In [42]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import REfunctions as REf

In [43]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import REfunctions as REf

mls_data = pd.read_csv("AgentResearch4865.csv")
# mls_data = mls_data.dropna(subset=['Baths']) 
# mls_data = mls_data.dropna(subset=['Beds']) 
mls_data.drop(mls_data[mls_data['Status'] == 'C/S'].index, inplace = True) 
REf.drop_zero_bed_bath(mls_data)
mls_data = mls_data[mls_data['Beds'] !=0]
REf.setdate(mls_data)
end_date =  mls_data['date'].max()
REf.get_end_date(mls_data)
mls_data = mls_data.rename(columns={'Current Price':'price',
                                    'List Price':'list_price',
                                    'Price/SqFt':'pricePsqft', 
                                    'Original List Price':'ogPrice',
                                    'Structure Type':'home_type',
                                    'Building Name':'building_name',
                                    'Selling Office Name':'selling_office',
                                    'Total SQFT':'total_sqft',
                                    'Year Built':'year_built',
                                    'Zip Code':'zip',
                                    'Legal Subdivision':'subdivision',
                                    'Above Grade Finished SQFT':'above_grade_sqft',
                                    'List Agent Full Name':'list_agent',
                                    'Selling Agent Full Name':'selling_agent',
                                    'List Office Name':'office'})

REf.fixbath(mls_data)
REf.fix_nums(mls_data)
REf.make_price_bins(mls_data)
REf.make_sqft_bins(mls_data)
REf.fix_status(mls_data)
REf.create_date_cols(mls_data)
REf.get_ab_rate(mls_data)
REf.make_col_absorption(mls_data)
# REf.process_df(mls_data,end_date)
days_back=30
a,m,act,clsd,cont,start,end =REf.get_ab_rate(mls_data)
print('Report Covers the latest',days_back,'days :',start.date(), 'to',end.date())
print('Current Active Listings :',act)
print('Closed Contracts over',days_back,'day period :',clsd)
print('Active Under Contract or Pending :',cont)
print(days_back, 'day absorbtion rate :',round(a,2),'%')
print(round((cont/(cont+act)),2)*100,'%','of listings are under Contract')
print('Months of inventory on hand :',m,'(i.e.', round(m*30,2), 'days)')

Report Covers the latest 30 days : 2021-03-21 to 2021-04-20
Current Active Listings : 38
Closed Contracts over 30 day period : 94
Active Under Contract or Pending : 1
30 day absorbtion rate : 241.0 %
3.0 % of listings are under Contract
Months of inventory on hand : 0.4 (i.e. 12.0 days)


In [45]:
mls_data[['year_month','space_bin','price']].groupby(['year_month','space_bin'], as_index=False).mean().head()

,year_month,space_bin,price
0,2015-06,A_0-1K,369228.222222
1,2015-06,B_1.0K-1.5K,571295.000000
2,2015-06,C_1.5k-2K,709590.000000
3,2015-06,D_2K+,948114.266667
4,2015-07,A_0-1K,335040.909091


In [46]:
df_last_30 = REf.last_x_days(mls_data,30)
df_last_30.head()

,MLS #,Cat,Status,Address,City,County,Beds,Baths,home_type,price,...,office,date,price_range,space_bin,month,year,year_month,m_y_bin,absorb,mon_inv
434,VAAX255852,CLS,CLS,719 S Saint Asaph St #206,Alexandria,"Alexandria City, VA",1.0,1,Unit/Flat/Apartment,225000.0,...,"Weichert, REALTORS (WEI58)",2021-03-24,$150k-$249k,A_0-1K,3,2021,2021-03,2021-03_$150k-$249k,150.0,0.33
487,VAAX250470,CLS,CLS,718 S Washington St #201,Alexandria,"Alexandria City, VA",1.0,1,Unit/Flat/Apartment,250000.0,...,Veach Realty Group (VEACH1),2021-04-01,$250k-$349k,A_0-1K,4,2021,2021-04,2021-04_$250k-$349k,200.0,0.43
503,VAAX257830,ACT,ACT,1718 W Abingdon Dr #301,Alexandria,"Alexandria City, VA",1.0,1.0,Unit/Flat/Apartment,276000.0,...,"EXP Realty, LLC (EXPY1)",2021-04-15,$250k-$349k,A_0-1K,4,2021,2021-04,2021-04_$250k-$349k,200.0,0.43
540,VAAX251828,CLS,CLS,501 Slaters Ln #213,Alexandria,"Alexandria City, VA",1.0,1.0,Unit/Flat/Apartment,255000.0,...,"McEnearney Associates, Inc. (MCE1)",2021-04-09,$250k-$349k,A_0-1K,4,2021,2021-04,2021-04_$250k-$349k,200.0,0.43
546,VAAX258236,ACT,ACT,1624 W Abingdon Dr #302,Alexandria,"Alexandria City, VA",1.0,1,Unit/Flat/Apartment,277000.0,...,"Long & Foster Real Estate, Inc. (LNG183)",2021-04-08,$250k-$349k,A_0-1K,4,2021,2021-04,2021-04_$250k-$349k,200.0,0.43


In [47]:
import altair as alt
# from altair_save import save

last_month_count = df_last_30[['Cat','price_range','price']].groupby(['Cat','price_range']).count()
last_month_count.reset_index(level=0, inplace=True)
last_month_count.reset_index(level=0, inplace=True)

last_month_count = last_month_count.rename(columns={'price_range':'Price Ranges',
                                                    'Cat':'Status',
                                                    'price':'Status Count'}) 
source = last_month_count

alt.Chart(source).mark_bar().encode(
    x='Status:O',
    y='Status Count:Q',
    color='Status:N',
    column='Price Ranges:N'
)



alt.Chart(...)

In [48]:
# month_count = mls_data[['m_y','space_bin','price']].groupby(['m_y','space_bin']).mean()
month_count = mls_data[['year_month','space_bin','price']].groupby(['year_month','space_bin']).mean()

month_count.reset_index(level=0, inplace=True)
month_count.reset_index(level=0, inplace=True)


print(month_count.head())


     space_bin year_month          price
0       A_0-1K    2015-06  369228.222222
1  B_1.0K-1.5K    2015-06  571295.000000
2    C_1.5k-2K    2015-06  709590.000000
3        D_2K+    2015-06  948114.266667
4       A_0-1K    2015-07  335040.909091


In [49]:
month_count = mls_data[['year_month','space_bin','price']].groupby(['year_month','space_bin']).mean()

month_count.reset_index(level=0, inplace=True)
month_count.reset_index(level=0, inplace=True)
# month_count['year_month'] = month_count['date'].dt.strftime("%Y-%m")

month_count = month_count.rename(columns={'space_bin':'SqFt Ranges',
                                                    'year_month':'Date',
                                                    'price':'Average Price'}) 
source = month_count

alt.Chart(source).mark_bar().encode(
    x='SqFt Ranges:N',
    y='Average Price:Q',
    color='SqFt Ranges:N',
    column='Date:O' 
)

alt.Chart(...)

In [50]:
month_count = mls_data[['year_month','space_bin','pricePsqft']].groupby(['year_month','space_bin']).mean()

month_count.reset_index(level=0, inplace=True)
month_count.reset_index(level=0, inplace=True)
# month_count['year_month'] = month_count['date'].dt.strftime("%Y-%m")

month_count = month_count.rename(columns={'space_bin':'SqFt Ranges',
                                                    'year_month':'Date',
                                                    'pricePsqft':'Avg_PirceperSqFt'}) 
source = month_count

alt.Chart(source).mark_bar().encode(
    x='SqFt Ranges:N',
    y='Avg_PirceperSqFt:Q',
    color='SqFt Ranges:N',
    column='Date:O' 
)

alt.Chart(...)

In [51]:

month_sale = mls_data[['price','date','space_bin','year','pricePsqft','year_month']]
# month_sale['year_month'] = month_sale['date'].dt.strftime("%Y-%m")
month_sale.head()


,price,date,space_bin,year,pricePsqft,year_month
1,325000.0,2015-09-09,NaN,2015,NaN,2015-09
2,330000.0,2017-06-08,NaN,2017,NaN,2017-06
3,350000.0,2015-12-21,NaN,2015,NaN,2015-12
4,351000.0,2016-03-30,NaN,2016,NaN,2016-03
5,358000.0,2016-01-22,NaN,2016,NaN,2016-01


In [52]:
pd.crosstab(mls_data.year_month, mls_data.space_bin,
          aggfunc=np.mean, values=mls_data.price)

space_bin,A_0-1K,B_1.0K-1.5K,C_1.5k-2K,D_2K+
year_month,,,,
2015-06,369228.222222,571295.000000,7.095900e+05,9.481143e+05
2015-07,335040.909091,532120.588235,6.480543e+05,1.131629e+06
2015-08,385620.000000,539515.909091,6.460000e+05,1.172282e+06
2015-09,346668.750000,591758.823529,6.322900e+05,1.020817e+06
2015-10,345453.846154,517266.666667,7.173778e+05,1.045025e+06
...,...,...,...,...
2020-12,384435.294118,755346.590909,8.392143e+05,1.935417e+06
2021-01,323245.062500,741334.916667,1.179591e+06,1.989154e+06
2021-02,406525.000000,735228.125000,9.096167e+05,1.300980e+06


In [53]:
month_smean = month_sale[['year_month','space_bin','pricePsqft']].groupby(['year_month','space_bin'], as_index=False).mean()
month_smean.head()

,year_month,space_bin,pricePsqft
0,2015-06,A_0-1K,465.820556
1,2015-06,B_1.0K-1.5K,465.993000
2,2015-06,C_1.5k-2K,456.428000
3,2015-06,D_2K+,429.225333
4,2015-07,A_0-1K,451.823636


In [54]:
month_pmean = month_sale[['year_month','space_bin','price']].groupby(['year_month','space_bin'], as_index=False).mean()

month_smean = month_sale[['year_month','space_bin','pricePsqft']].groupby(['year_month','space_bin'], as_index=False).mean()


# month_sale = mls_data[['price','date','space_bin','year']]
chart1 = alt.Chart(month_pmean).mark_circle().encode(
    x='year_month:T',
    y='price:Q',
    color='space_bin:N'
).properties(
    height=300,
    width=300
)
# + base.transform_loess('year_month', 'y', groupby=['size']).mark_line(size=4)
#
chart2 = alt.Chart(month_pmean).mark_bar().encode(
    x='count()',
    y=alt.Y('price:Q',bin=alt.Bin(maxbins=30)),
    color='space_bin:N'
).properties(
    height=300,
    width=100
)

chart1 | chart2
# chart1 + chart1.transform_loess('year_month:T', 'price:Q',groupby=np.array(sqft_bins)).mark_line(size=4)


alt.HConcatChart(...)

In [55]:
month_sale = mls_data[['price','date','space_bin','year','pricePsqft','year_month']]

line = alt.Chart(month_sale).mark_line().encode(
    x='year_month',
    y='mean(price)'
).properties(
    height=300,
    width=800
)

band = alt.Chart(month_sale).mark_errorband(extent='ci').encode(
    x='year_month',
    y=alt.Y('price', title='Price Range'),
).properties(
    height=300,
    width=800
)

band + line

alt.LayerChart(...)

In [56]:
from vega_datasets import data

source = data.iris()

alt.Chart(source).transform_fold(
    ['petalWidth', # sqft bins
     'petalLength',
     'sepalWidth',
     'sepalLength'],
    as_ = ['Measurement_type', 'value']
).transform_density(
    density='value', # total sqft
    bandwidth=0.3,
    groupby=['Measurement_type'], # sqft bin
    extent= [0, 8]
).mark_area().encode(
    alt.X('value:Q'), #sqft
    alt.Y('density:Q'),
    alt.Row('Measurement_type:N') 
).properties(width=300, height=50)

alt.Chart(...)

In [57]:
data.iris().head()

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [58]:
sale_density = mls_data[['price','date','price_range','space_bin','year','pricePsqft','year_month','total_sqft']]
# sale_density.head()

In [59]:
month_pmean.head()

,year_month,space_bin,price
0,2015-06,A_0-1K,369228.222222
1,2015-06,B_1.0K-1.5K,571295.000000
2,2015-06,C_1.5k-2K,709590.000000
3,2015-06,D_2K+,948114.266667
4,2015-07,A_0-1K,335040.909091


In [60]:
test1 = sale_density[['year_month','space_bin','price']].groupby(['year_month','space_bin']).mean()
test1.reset_index(level=0, inplace=True)
test1.reset_index(level=0, inplace=True)

test2=pd.crosstab(mls_data.year_month, mls_data.space_bin,
          aggfunc=np.mean, values=mls_data.price)

sourceMine = pd.DataFrame({
    'x': test2.index,
    'A_low': list(test2['A_0-1K']),
    'B': list(test2['B_1.0K-1.5K']),
    'C': list(test2['C_1.5k-2K']),
    'D': list(test2['D_2K+']),
})

base = alt.Chart(sourceMine).mark_circle(opacity=0.5).transform_fold(
    fold=['A_low', 'B', 'C','D'],
    as_=['category', 'y']
).encode(
    alt.X('x:T'),
    alt.Y('y:Q'),
    alt.Color('category:N')
)

chart2 = alt.Chart(month_pmean).mark_bar().encode(
    x='count()',
    y=alt.Y('price:Q',bin=alt.Bin(maxbins=25)),
    color='space_bin:N'
).properties(
    height=300,
    width=100
)

# base
base + base.transform_loess('x', 'y', groupby=['category'],bandwidth=.4).mark_line(size=4) | chart2

alt.HConcatChart(...)

In [61]:
test2

space_bin,A_0-1K,B_1.0K-1.5K,C_1.5k-2K,D_2K+
year_month,,,,
2015-06,369228.222222,571295.000000,7.095900e+05,9.481143e+05
2015-07,335040.909091,532120.588235,6.480543e+05,1.131629e+06
2015-08,385620.000000,539515.909091,6.460000e+05,1.172282e+06
2015-09,346668.750000,591758.823529,6.322900e+05,1.020817e+06
2015-10,345453.846154,517266.666667,7.173778e+05,1.045025e+06
...,...,...,...,...
2020-12,384435.294118,755346.590909,8.392143e+05,1.935417e+06
2021-01,323245.062500,741334.916667,1.179591e+06,1.989154e+06
2021-02,406525.000000,735228.125000,9.096167e+05,1.300980e+06


In [62]:
source = pd.DataFrame({
    'x': test2.index,
    'A': list(test2['A_0-1K']),
    'B': list(test2['B_1.0K-1.5K']),
    'C': list(test2['C_1.5k-2K']),
    'D': list(test2['D_2K+']),
})

alt.Chart(source).transform_fold(
    ['A','B','C','D'],
    as_ = ['Measurement_type', 'value']
).transform_density(
    density='value', # total sqft
    bandwidth=100000,
    groupby=['Measurement_type'], # sqft bin
    extent= [400000, 2000000]
).mark_area().encode(
    alt.X('value:Q'), #sqft
    alt.Y('density:Q'),
    alt.Row('Measurement_type:N') 
).properties(width=300, height=50)

alt.Chart(...)

In [63]:
sourceMine.head()

,x,A_low,B,C,D
0,2015-06,369228.222222,571295.000000,709590.000000,9.481143e+05
1,2015-07,335040.909091,532120.588235,648054.285714,1.131629e+06
2,2015-08,385620.000000,539515.909091,646000.000000,1.172282e+06
3,2015-09,346668.750000,591758.823529,632290.000000,1.020817e+06
4,2015-10,345453.846154,517266.666667,717377.777778,1.045025e+06


In [38]:
source.head()

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [39]:
month_pmean.head()
month_pmean['dateT'] = pd.to_datetime(month_pmean.year_month,format="%Y-%m")
month_pmean.head()


,year_month,space_bin,price,dateT
0,2015-06,A_0-1K,369228.222222,2015-06-01
1,2015-06,B_1.0K-1.5K,571295.000000,2015-06-01
2,2015-06,C_1.5k-2K,709590.000000,2015-06-01
3,2015-06,D_2K+,948114.266667,2015-06-01
4,2015-07,A_0-1K,335040.909091,2015-07-01


In [40]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(month_pmean, x="dateT", y="price", color="space_bin", marginal_y="box",
            trendline="ols", template="simple_white")
fig.show()

In [41]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.plotting import ColumnDataSource

data = ColumnDataSource(df_last_30)

plot = figure(x_axis_type='datetime',x_axis_label = 'Date', y_axis_label='Sales')

plot.circle(x='date',y='price',source=data, color='blue', width=0.5)
# output_file('pandas_bins.html')
show(plot)

